In [ ]:
#try the stacking method in this link
#https://www.kaggle.com/code/arthurtok/introduction-to-ensembling-stacking-in-python#Generating-our-Base-First-Level-Models

In [1]:
from wcds import *

In [2]:
df_train = pd.read_csv(r"D:\data science data\abalone\train.csv", index_col='id')
df_test =   pd.read_csv(r"D:\data science data\abalone\test.csv", index_col='id')
df_train.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,,
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [3]:
df_test.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
id,,,,,,,,
90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050


In [4]:
df_original = pd.read_csv(r"D:\data science data\abalone\abalone.csv")
df_original.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
df_original.shape

(4177, 9)

In [6]:
#concat orginal and train
df_train_merge = pd.concat([df_train, df_original.rename(columns={'Viscera weight':"Whole weight.2", 'Shucked weight':'Whole weight.1'})], ignore_index=True).drop_duplicates()


In [7]:
df_train_merge.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1',
       'Whole weight.2', 'Shell weight', 'Rings'],
      dtype='object')

In [32]:
X_train = df_train_merge.drop(['Rings' ], axis =1)
y_train = df_train_merge['Rings']


In [9]:
def add_feature (df, max_ratio, shell_volume_flag, meat_shell_flag, shell_surface_flag, proportional_weight_flag, dimension_ratio_flag ):
    df = df.copy()
    #shell volume flag
    if shell_volume_flag:
        df['Shell_Volume'] = df['Length'] * df['Diameter'] * df['Height']

    # Meat to shell ratio
    if meat_shell_flag:
        df['Meat_to_Shell_Ratio'] = df['Whole weight.1'] / df['Shell weight']

    # Shell surface area
    if shell_surface_flag:
        df['Shell_Surface_Area'] = 2 * ((df['Length'] * df['Diameter']) + (df['Length'] * df['Height']) + (df['Diameter'] * df['Height']))

    # Proportional weights
    if proportional_weight_flag:
        df['Whole_to_Shell_Ratio'] = df['Whole weight'] / df['Shell weight']
        df['Shucked_to_Whole_Ratio'] = df['Whole weight.1'] / df['Whole weight']
        df['Viscera_to_Whole_Ratio'] = df['Whole weight.2'] / df['Whole weight']

    # Dimension ratios
    if dimension_ratio_flag:
        df['Length_to_Diameter_Ratio'] = df['Length'] / df['Diameter']
        df['Length_to_Height_Ratio'] = df['Length'] / df['Height']
        df['Length_to_Height_Ratio'] = df['Length_to_Height_Ratio'].apply(lambda df: min(df, max_ratio))

        df['Diameter_to_Height_Ratio'] = df['Diameter'] / df['Height']
        df['Diameter_to_Height_Ratio'] = df['Diameter_to_Height_Ratio'].apply(lambda df: min(df, max_ratio))

    return df

In [10]:
from joblib import load
pipe_xgb = load(r"D:\data science data\abalone\xgb_pipe.pkl")
pipe_xgb

Pipeline(steps=[('add feature',
                 FunctionTransformer(func=<function add_feature at 0x0000023020B6C7C0>,
                                     kw_args={'dimension_ratio_flag': 0,
                                              'max_ratio': 1,
                                              'meat_shell_flag': 0,
                                              'proportional_weight_flag': 0,
                                              'shell_surface_flag': 0,
                                              'shell_volume_flag': 0})),
                ('preprocess',
                 FeatureUnion(transformer_list=[('pipeline-1',
                                                 Pipeline(steps=[('excludecolumnselector',
                                                                  ExcludeC...
                              gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.00530171011345918, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=18,
                              max_leaves=None, min_child_weight=13, missing=nan,
                              monotone_constraints=None, n_estimators=831,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [11]:
pipe_xgb.predict(df_test)

array([ 9.937871, 10.094482,  9.467059, ..., 11.940689, 12.917361,
        8.516136], dtype=float32)

In [12]:
pipe_lgb = load(r"D:\data science data\abalone\lgb_pipe.pkl")
pipe_cat = load(r"D:\data science data\abalone\cat_pipe.pkl")

In [14]:
from sklearn.model_selection import KFold

In [17]:
ntrain = X_train.shape[0]
ntest = df_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold( n_splits= NFOLDS, shuffle=True,  random_state=SEED)

In [18]:
# Wrapper Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [20]:
np.empty((NFOLDS, 10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [27]:
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    print (i, (train_index, test_index))

0 (array([    0,     1,     2, ..., 94788, 94789, 94791]), array([    3,     6,     9, ..., 94780, 94785, 94790]))
1 (array([    0,     2,     3, ..., 94788, 94789, 94790]), array([    1,     8,    11, ..., 94783, 94786, 94791]))
2 (array([    1,     2,     3, ..., 94789, 94790, 94791]), array([    0,     4,     5, ..., 94778, 94787, 94788]))
3 (array([    0,     1,     2, ..., 94788, 94790, 94791]), array([    7,    16,    19, ..., 94782, 94784, 94789]))
4 (array([    0,     1,     3, ..., 94789, 94790, 94791]), array([    2,    10,    13, ..., 94760, 94767, 94771]))


In [36]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)   #this is test group in train set, assign predict value
        oof_test_skf[i, :] = clf.predict(x_test)    # use model to predict test, this is 2d array, contain nfold

    oof_test[:] = oof_test_skf.mean(axis=0)  #this get average prediction of df_test
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)   #get train, and test prediction 


In [23]:
# y_train = y_train.ravel()
# train = X_train
# x_train = train.values # Creates an array of the train data
# x_test = df_test.values 


In [35]:
X_train.iloc[np.array([0, 2, 3])]

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.240
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.005
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.250


In [37]:
xgb_oof_train, xgb_oof_test = get_oof(pipe_xgb, X_train, y_train, df_test) # xgb
lgb_oof_train, lgb_oof_test = get_oof(pipe_lgb,X_train, y_train, df_test) # lgb
cat_oof_train, cat_oof_test = get_oof(pipe_cat, X_train, y_train, df_test) # cat


print("Training is complete")

0:	learn: 2.9893385	total: 190ms	remaining: 3m
1:	learn: 2.8279866	total: 200ms	remaining: 1m 34s
2:	learn: 2.6858036	total: 206ms	remaining: 1m 5s
3:	learn: 2.5703045	total: 213ms	remaining: 50.3s
4:	learn: 2.4691350	total: 220ms	remaining: 41.7s
5:	learn: 2.3844271	total: 227ms	remaining: 35.7s
6:	learn: 2.3113392	total: 234ms	remaining: 31.5s
7:	learn: 2.2494194	total: 240ms	remaining: 28.3s
8:	learn: 2.1988636	total: 247ms	remaining: 25.8s
9:	learn: 2.1563168	total: 253ms	remaining: 23.8s
10:	learn: 2.1193026	total: 259ms	remaining: 22.1s
11:	learn: 2.0895116	total: 266ms	remaining: 20.8s
12:	learn: 2.0640131	total: 272ms	remaining: 19.6s
13:	learn: 2.0422775	total: 278ms	remaining: 18.6s
14:	learn: 2.0249469	total: 284ms	remaining: 17.7s
15:	learn: 2.0086400	total: 290ms	remaining: 16.9s
16:	learn: 1.9954260	total: 297ms	remaining: 16.3s
17:	learn: 1.9847249	total: 303ms	remaining: 15.7s
18:	learn: 1.9757161	total: 310ms	remaining: 15.2s
19:	learn: 1.9682282	total: 318ms	remaining

In [38]:
x_train = np.concatenate(( xgb_oof_train, lgb_oof_train,cat_oof_train), axis=1)
x_test = np.concatenate(( xgb_oof_test, lgb_oof_test,cat_oof_test), axis=1)

In [39]:
import xgboost
#test if I don't use target transform
def objective(trial):
    

    xgb_param = {
        
        'eval_metric': 'rmse',
        'random_state': 42,
        'objective': 'reg:squarederror',
        'gamma': trial.suggest_float("gamma", 1e-2, 1.0),
        'max_depth': trial.suggest_int('max_depth',2, 20),
        'subsample': trial.suggest_float("subsample", 0.05, 1.0),
        'n_estimators': trial.suggest_int('n_estimators',100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight',2, 20),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.05, 1.0),
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),

    }

    model = xgboost.XGBRegressor( **xgb_param)

    
    skf = StratifiedKFold(n_splits=5) 
    
    cv_score = cross_val_score(model, x_train, y_train, n_jobs=4, cv=skf, scoring='neg_mean_squared_log_error',verbose=1, error_score='raise')

    return cv_score.mean()



study = optuna.create_study(direction="maximize") # maximise the score during tuning
study.optimize(objective, n_trials=100) # run the objective function 100 times

print(study.best_trial) # print the best performing pipeline


[I 2024-05-01 22:02:13,064] A new study created in memory with name: no-name-0895b6ee-54d1-4d84-ada2-3753426f19e8
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   15.8s finished
[I 2024-05-01 22:02:29,038] Trial 0 finished with value: -0.022169308234905406 and parameters: {'gamma': 0.2068573766282819, 'max_depth': 2, 'subsample': 0.05779036567053651, 'n_estimators': 699, 'min_child_weight': 7, 'colsample_bytree': 0.5595093723865857, 'learning_rate': 0.008379015545505809}. Best is trial 0 with value: -0.022169308234905406.
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   39.7s finished
[I 2024-05-01 22:03:08,912] Trial 1 finished with value: -0.022149206531477138 and parameters: {'gamma': 0.32721094632122905, 'max_depth': 3, 'subsample': 0.8074848974480889, 'n_estimators': 729, 'min_child_weight': 2, 'colsample_bytree': 0.92914

FrozenTrial(number=22, values=[-0.022019310926042646], datetime_start=datetime.datetime(2024, 5, 1, 22, 20, 22, 210798), datetime_complete=datetime.datetime(2024, 5, 1, 22, 21, 2, 677866), params={'gamma': 0.3484025318375001, 'max_depth': 4, 'subsample': 0.6230862937295858, 'n_estimators': 650, 'min_child_weight': 7, 'colsample_bytree': 0.777126613189779, 'learning_rate': 0.007020575494601785}, distributions={'gamma': UniformDistribution(high=1.0, low=0.01), 'max_depth': IntUniformDistribution(high=20, low=2, step=1), 'subsample': UniformDistribution(high=1.0, low=0.05), 'n_estimators': IntUniformDistribution(high=1000, low=100, step=1), 'min_child_weight': IntUniformDistribution(high=20, low=2, step=1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.05), 'learning_rate': LogUniformDistribution(high=0.1, low=0.001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=22, state=TrialState.COMPLETE, value=None)


In [40]:
model = xgboost.XGBRegressor( **study.best_params)
model.fit(x_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.777126613189779, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.3484025318375001, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.007020575494601785, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             n_estimators=650, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [43]:
def get_submission(pipe1, X, y, x_test, df_test, file_name):
    pipe1.fit(X, y)
    y_predict = pipe1.predict(x_test)
    #create df 
    df_final = pd.DataFrame()
    df_final['id'] = df_test.index
    df_final['Rings'] = pd.Series(y_predict)
    save_path = os.path.join(r"D:\data science data\abalone", f"{file_name}.csv")
    df_final.to_csv(save_path, index=False)
    return

In [44]:
get_submission(model, x_train, y_train, x_test, df_test, 'submission15_stacking_final')